# Greedy für max weighted matching

Wir illustrieren hier wie ein greedy-Algorithmus im Problem der Paarungen maximalen Gewichts formuliert werden kann. 

Zunächst brauchen wir ein paar *imports*. Wenn Sie unter Colab arbeiten, sind diese Pakete bereits installiert. Wenn Sie lokal auf Ihrem eigenen Rechner arbeiten, müssen Sie gegebenenfalls einige der Pakete installieren, und zwar per <code>pip install numpy</code> oder <code>conda install numpy</code>.

In [1]:
import numpy as np  ## https://numpy.org Python-Bibliothek für wissenschaftliches Rechnen 
import random # Zufallszahlen

import matplotlib.pyplot as plt # Visualisierung
from IPython.display import clear_output # Interaktion mit jupyter
import ipywidgets as widgets # Interaktion mit jupyter

Als erstes erzeugen wir Zufallsinstanzen. Die Instanz selbst ist als Matrizen kodiert. 

In [2]:
def random_instance(left_size,right_size,edge_proba):
    """
    generates random instances with profits from 1 to 100 (or 0 if row/column combination is not feasible).
    left_size, right_size: number of rows and columns in profit matrix
    edge_proba: probability that some row/column combination has positive profit.
    """
    profits=np.zeros((left_size,right_size))  # initialise profits to 0
    for l in range(left_size):
        for r in range(right_size):
            if random.random()<=edge_proba:   # do random experiment to see whether row/column feasible
                profits[l,r]=random.randint(1,100)  # if yes, draw random profit
    return profits

profits=random_instance(4,5,0.5)
profits

array([[ 0.,  0., 98.,  0.,  0.],
       [ 0.,  3., 58., 27., 55.],
       [ 8.,  4., 55.,  0.,  0.],
       [ 0.,  0.,  0., 33.,  0.]])

Der greedy-Algorithmus ist denkbar einfach: in jeder Runde wählen wir unter den Zeile/Spalten-Kombinationen, die noch möglich sind, diejenige mit höchstem Profit aus. Um nicht in jeder Runde neu nach der Kombination mit höchstem Profit suchen zu müssen, werden die Zeile/Spalten-Paare einmal am Anfang nach Profit sortiert. Schließlich muss nur noch gewährleistet werden, dass wir uns die Zeilen und Spalten merken, die bereits benutzt wurden.

Die Ausgabe besteht aus der Zuordnung <code>assignment</code>, eine Liste von Zeile/Spalte-Paaren <code>(i,j)</code>. Dies sind gerade die ausgewählten Einträge der Profitmatrix.

In [3]:
def greedy_max_matching(profits,return_history=False):
    """
    expects profit matrix (numpy array) as input, all entries should be non-negative
    outputs: total profit, assignment
    where assignment is a list of pairs (i,j) meaning that row i is matched with column j
    """
    total_profit=0
    assignment=[]
    history=[([],[],[])]
    L,R=profits.shape  # L-> number of rows, R-> number of columns
    used_left,used_right=[],[]  # keep track of which rows/columns are already matched
    potential_profits=[(profits[l,r],l,r) for l in range(L) for r in range(R) if profits[l,r]>0]
    potential_profits.sort(reverse=True)  # sort row/column pairs by profit, highest first
    for profit,l,r in potential_profits:
        if not l in used_left and not r in used_right:  # if row/column still feasible, take it
            used_left.append(l)     # row becomes infeasible
            used_right.append(r)    # column becomes infeasible
            assignment.append((l,r)) # keep track of assignment
            total_profit+=profit    # keep track of profit
            if return_history:
                history.append((assignment.copy(),used_left.copy(),used_right.copy()))
    if return_history:
        return total_profit,assignment,history 
    return total_profit,assignment

In [5]:
profit,assignment=greedy_max_matching(profits)
print("Profit: {}".format(profit))
print("Paarung: ")
print(assignment)

Profit: 194.0
Paarung: 
[(0, 2), (1, 4), (3, 3), (2, 0)]


## Visualisierung

Hier soll der Verlauf des greedy-Algorithmus noch visualisiert werden. Der Code ist ein wenig komplex, muss aber nicht verstanden werden. 

In [6]:
def compute_profits(profits,assignment):
    return sum([profits[i,j] for i,j in assignment])

def show_matrix(profits,assignment,used_left=[],used_right=[]):
    fig,ax=plt.subplots(figsize=(5,5))

    ax.axis("off")
    ax.set_xlim(0,10)
    ax.set_ylim(0,10)
    ax.set_title("Profit: {}".format(compute_profits(profits,assignment)))
    m,n=profits.shape

    scalex=9.5/n
    scaley=9.5/m
    offsetx=0+0.25*scalex
    offsety=0+0.25*scaley

    for i in range(m-1,-1,-1):
        for j in range(n):
            color='black'
            fontweight='normal'
            if (i,j) in assignment:
                color='tab:blue'
                fontweight="bold"
            elif (i in used_left) or (j in used_right):
                color='gray'
            ax.text(x=j*scalex+offsetx,y=i*scaley+offsety,s=round(profits[i,j]),  
                ha='center', va='center', color=color,size=12,fontweight=fontweight)

            
def show_movie(profits, history,sleep_after=2):
  for assignment,used_left,used_right in history:
    clear_output(wait=True)
    show_matrix(profits,assignment,used_left=used_left,used_right=used_right)
    plt.pause(sleep_after)

#### Code für Widgets / Startbutton usw
go_button=widgets.Button(
    description='Start',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    icon="play",
    tooltip='Startet Visualisierung')

output = widgets.Output()
animation_widget=widgets.VBox([go_button,output])

def go(event):
    total_profit,assignment,history = greedy_max_matching(profits,return_history=True)
    with output:
        show_movie(profits,history)
        
go_button.on_click(go)

In [7]:
profits=random_instance(7,9,0.5)


animation_widget